# &nbsp; Project: Gans company 

## 1.&nbsp; Scraping Basic Information

We need to import some functions:

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from lat_lon_parser import parse    # for decimal coordinates
from datetime import datetime # to get today's date

You will need to install sqlalchemy and pymysql, if they are not already installed.

In [3]:
# install if needed
# !pip install sqlalchemy
# !pip install pymysql

### 1.1.&nbsp; Scraping country, latitude and longitude of each city
We scrape the basic information of each city from their respective Wikipedia pages:

 - Berlin: https://en.wikipedia.org/wiki/Berlin
 - Hamburg: https://en.wikipedia.org/wiki/Hamburg
 - Munich: https://en.wikipedia.org/wiki/Munich

In [4]:
list_of_cities = ["Berlin", "Hamburg", "Munich"]

We write a function to scrape the country, latitude and longitude of each city and make a datafrmae of it:

In [5]:
def cities_dataframe(cities):

  city_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

    # keep track of data per city
    city_data.append({"City": city,
                    "Country": country,
                    "Latitude": parse(city_latitude), # latitude in decimal format
                    "Longitude": parse(city_longitude), # longitude in decimal format
                    })

  return pd.DataFrame(city_data)

In [6]:
cities_df = cities_dataframe(list_of_cities)
cities_df

,City,Country,Latitude,Longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


In [7]:
# Check the data type of all columns.
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       3 non-null      object 
 1   Country    3 non-null      object 
 2   Latitude   3 non-null      float64
 3   Longitude  3 non-null      float64
dtypes: float64(2), object(2)
memory usage: 228.0+ bytes


### 1.2.&nbsp; Scraping population
Then, we extract the population of a city. Since, this information changes over time, we also need to add a timestamp.

In [8]:
def populations_dataframe(cities):

    population_data = []

    for city in cities:
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

        # extract the relevant information
        city_population = city_soup.find(string="Population").find_next("td").get_text()
        city_population_clean = int(city_population.replace(",", ""))
        today = datetime.today().strftime("%d.%m.%Y")
        today_clean = datetime.strptime(today, "%d.%m.%Y")

        # keep track of data per city
        population_data.append({"City": city,
                        "Population": city_population_clean,
                        "Timestamp_Population": today_clean
                        })

    return pd.DataFrame(population_data)

In [9]:
population_df = populations_dataframe(list_of_cities)
population_df

,City,Population,Timestamp_Population
0,Berlin,3878100,2024-12-08
1,Hamburg,1964021,2024-12-08
2,Munich,1510378,2024-12-08


In [10]:
# Check the data type of all columns again.
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   City                  3 non-null      object        
 1   Population            3 non-null      int64         
 2   Timestamp_Population  3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 204.0+ bytes


---
## 2.&nbsp; Accessing the data using APIs with keys


### 2.1.&nbsp; Collect weather data using the [OpenWeather](https://openweathermap.org/).
We creating a function for multiple cities to get the weather information and make a datafrmae of it:

In [11]:
# Define the API key to be passed into the url in a .py file called safe.
from safe import API_key

In [12]:
def weather_dataframe(list_of_cities):
  
  # Define the limit to be passed into the url.
  limit = 5

  # Create an empty list for each of the values.
  weather_dictionary = {"city_name": [],
                        "forecast_time": [],
                        "outlook": [],
                        "description": [],
                        "temperature": [],
                        "feels_like": [],
                        "temp_min": [],
                        "temp_max": [],
                        "wind_speed": [],
                        "rain_prob": []}
  
  # Get the coordinates of the desired city
  for city in list_of_cities:
    coordinates = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&limit={limit}&appid={API_key}")
    coordinates_json= coordinates.json()

    # Define the parameters to be passed into the url.
    lat = coordinates_json["city"]["coord"]["lat"]
    lon = coordinates_json["city"]["coord"]["lon"]

    # Define the parameters to be passed into the url.
    weather = (requests
           .get(
               f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric"
               )
               )
    weather_json = weather.json()

    # Iterate over the list inside the weather_dictionary to get the data for different days and time.
    i = 0

    for _ in weather_json["list"]:
      # extract the relevant information
      weather_dictionary["city_name"].append(weather_json["city"]["name"])
      weather_dictionary["forecast_time"].append(weather_json["list"][i]["dt_txt"])
      weather_dictionary["outlook"].append(weather_json["list"][i]["weather"][0]["main"])
      weather_dictionary["description"].append(weather_json["list"][i]["weather"][0]["description"])
      weather_dictionary["temperature"].append(weather_json["list"][i]["main"]["temp"])
      weather_dictionary["feels_like"].append(weather_json["list"][i]["main"]["feels_like"])
      weather_dictionary["temp_min"].append(weather_json["list"][i]["main"]["temp_min"])
      weather_dictionary["temp_max"].append(weather_json["list"][i]["main"]["temp_max"])
      weather_dictionary["wind_speed"].append(weather_json["list"][i]["wind"]["speed"])
      weather_dictionary["rain_prob"].append(weather_json["list"][i]["pop"])
      i += 1

  weather_dict = pd.DataFrame(weather_dictionary)
  weather_dict["forecast_time"] = pd.to_datetime(weather_dict["forecast_time"])

  return (weather_dict)

In [13]:
weather_df = weather_dataframe(list_of_cities)

In [14]:
weather_df.sample(5)

,city_name,forecast_time,outlook,description,temperature,feels_like,temp_min,temp_max,wind_speed,rain_prob
93,Munich,2024-12-10 09:00:00,Snow,light snow,0.83,-1.95,0.83,0.83,2.41,0.2
58,Hamburg,2024-12-11 03:00:00,Clouds,overcast clouds,1.57,-0.26,1.57,1.57,1.71,0.0
49,Hamburg,2024-12-10 00:00:00,Clouds,overcast clouds,5.54,1.84,5.54,5.54,5.39,0.0
64,Hamburg,2024-12-11 21:00:00,Clouds,overcast clouds,1.28,-1.00,1.28,1.28,2.03,0.0
72,Hamburg,2024-12-12 21:00:00,Clouds,overcast clouds,3.94,2.97,3.94,3.94,1.34,0.0


In [15]:
# Check the data type of all columns.
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   city_name      120 non-null    object        
 1   forecast_time  120 non-null    datetime64[ns]
 2   outlook        120 non-null    object        
 3   description    120 non-null    object        
 4   temperature    120 non-null    float64       
 5   feels_like     120 non-null    float64       
 6   temp_min       120 non-null    float64       
 7   temp_max       120 non-null    float64       
 8   wind_speed     120 non-null    float64       
 9   rain_prob      120 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 9.5+ KB


In [16]:
#city_mask1 = weather_df["city_name"] == "Altstadt"
#weather_df.loc[city_mask1, 'city_name'] = "Munich"

#city_mask2 = weather_df["city_name"] == "Mitte"
#weather_df.loc[city_mask2, 'city_name'] = "Berlin"

### 2.2.&nbsp; Collect airports and flights data from [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/).

#### 2.2.1.&nbsp; Accessing airports information
We create a function to collect airports information based on the latitudes and longitudes from the cities dataframe:

In [17]:
latitudes = list(cities_df.Latitude)
longitudes = list(cities_df.Longitude)

In [ ]:
from safe import API_key_AeroDataBox

In [18]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)

  list_for_df = []

  for index, value in enumerate(latitudes):

    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    
    querystring = {"lat":{value},"lon":{longitudes[index]},"radiusKm":"95","limit":"10","withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": {API_key_AeroDataBox}
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    # Correctly access the 'items' key from the JSON response
    items = response.json()['items']
    list_for_df.append(pd.json_normalize(items))

  airports = pd.concat(list_for_df, ignore_index=True)
  airports = (airports
            .drop(
                columns=["iata", "shortName", "countryCode", "timeZone", "location.lat", "location.lon"]
                , axis=1)
                )
  airports = (airports
              .rename(
                columns={'icao': 'airport_icao', 'name': 'airport_name'}
                )
                )

  return (airports)

In [19]:
airports = icao_airport_codes(latitudes, longitudes)
airports

,airport_icao,airport_name,municipalityName
0,EDDB,Berlin Brandenburg,Berlin
1,EDDH,Hamburg,Hamburg
2,EDDM,Munich,Munich


In [20]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   airport_icao      3 non-null      object
 1   airport_name      3 non-null      object
 2   municipalityName  3 non-null      object
dtypes: object(3)
memory usage: 204.0+ bytes


#### 2.2.2.&nbsp; Accessing flights information
We create a function to collect flights information based on the list of icao from the airports dataframe:

In [21]:
icao_list = list(airports.airport_icao)

In [22]:
icao_list

['EDDB', 'EDDH', 'EDDM']

In [23]:
def flight_dataframe(icao_list):
    
    # Create an empty list for each of the values.
    flight_dict_items = []

    # Get the icao of the desired airports
    for icaos in icao_list:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icaos}/2024-12-04T08:00/2024-12-04T20:00"
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        headers = {
            "x-rapidapi-key": {API_key_AeroDataBox},
            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }
        response = requests.get(url, headers=headers, params=querystring)
        flight_json = response.json()

        for item in flight_json["arrivals"]:
            flight_dict_item = {
                "icao": icaos,
                "icao_arrival": item["movement"]["airport"].get("icao", None),
                "arrival_local_time": item["movement"]["scheduledTime"].get("local", None),
                "arrival_utc_time": item["movement"]["scheduledTime"].get("utc", None),
                "revised_local_time": item["movement"].get("revisedTime", {}).get("local", 0),
                "revised_utc_time": item["movement"].get("revisedTime", {}).get("utc", 0),
                "flight_number": item.get("number", None),
                "flight_status": item.get("status", None),
                "airline_name": item["airline"].get("name", None),
                }
            
            flight_dict_items.append(flight_dict_item)
        
    flight_dict_df = pd.DataFrame(flight_dict_items)
    flight_dict_df["arrival_local_time"] = pd.to_datetime(flight_dict_df["arrival_local_time"])
    # Remove the timezone
    flight_dict_df["arrival_local_time"] = flight_dict_df["arrival_local_time"].dt.tz_localize(None)
    # Add missing seconds before conversion
    flight_dict_df["revised_local_time"] = (flight_dict_df["revised_local_time"]
                                           .str
                                           .replace(r"(?<=\d{2}:\d{2})(?=\+)", ":00", regex=True)
                                           )
    # Convert to datetime
    flight_dict_df["revised_local_time"] = pd.to_datetime(flight_dict_df["revised_local_time"])
    # Remove the timezone
    flight_dict_df["revised_local_time"] = flight_dict_df["revised_local_time"].dt.tz_localize(None)
    #we can remove some columns if we don't need them
    flight_dict_df = (flight_dict_df
                      .drop(
                          columns=["arrival_utc_time","revised_utc_time","airline_name"]
                          )
                          )

    return (flight_dict_df)

In [24]:
flight_df = flight_dataframe(icao_list)

In [25]:
flight_df.sample(3)

,icao,icao_arrival,arrival_local_time,revised_local_time,flight_number,flight_status
503,EDDM,EDDV,2024-12-04 18:20:00,2024-12-04 18:05:00,VL 2101,Arrived
421,EDDM,LSZH,2024-12-04 14:30:00,2024-12-04 14:35:00,LH 2369,Arrived
1,EDDB,EDDR,2024-12-04 08:05:00,2024-12-04 08:11:00,DX 121,Arrived


In [26]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   icao                566 non-null    object        
 1   icao_arrival        558 non-null    object        
 2   arrival_local_time  566 non-null    datetime64[ns]
 3   revised_local_time  566 non-null    datetime64[ns]
 4   flight_number       566 non-null    object        
 5   flight_status       566 non-null    object        
dtypes: datetime64[ns](2), object(4)
memory usage: 26.7+ KB


---
## 3.&nbsp; Sending & retrieving the information from this notebook to sql and from sql to this notebook
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [27]:
# Save your password in a .py file called safe.
from safe import sql_password

In [28]:
schema = "CityDatabase"
host = "127.0.0.1"
user = "root"
password = sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### 3.1.&nbsp; cities_df

#### 3.1.1.&nbsp; Create "cities_info" table in the database

```sql
CREATE TABLE cities_info (
	city_id INT AUTO_INCREMENT, -- Automatically generated ID for each city
    City VARCHAR(255) NOT NULL, -- Name of the city
    Country VARCHAR(255) NOT NULL, -- Name of the Country
    Latitude DECIMAL(6,4), -- Latitude of the city
    Longitude DECIMAL(5,3), -- Longitude of the city
    PRIMARY KEY (city_id) -- Primary key to uniquely identify each city
);
```

#### 3.1.2.&nbsp; Push the "cities_df" to the empty "cities_info" table in the database

In [ ]:
cities_df.to_sql('cities_info',
                 if_exists='append',
                  con=connection_string,
                  index=False)

#### 3.1.3.&nbsp; Retrieving the information from sql to this notebook

In [29]:
city_info_from_sql = pd.read_sql("cities_info", con=connection_string)
city_info_from_sql

,city_id,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


### 3.2.&nbsp; population_df

#### 3.2.1.&nbsp; Preparing the population table
By extracting the cities_info table from our SQL database, we gain access to the unique identifier `city_id` assigned to each city. These `city_id`'s serve as pointers to their corresponding city records, allowing us to seamlessly link the `city_id`'s in the cities_info to their respective city in the population table, thereby completing the population table.

In [50]:
population_city_df = population_df.merge(city_info_from_sql,
                                   on = "City",
                                   how="left")

population_city_df

,City,Population,Timestamp_Population,city_id,Country,Latitude,Longitude
0,Berlin,3878100,2024-12-05,1,Germany,52.5200,13.405
1,Hamburg,1964021,2024-12-05,2,Germany,53.5500,10.000
2,Munich,1510378,2024-12-05,3,Germany,48.1375,11.575


In [51]:
population_city_df = (population_city_df
                      .drop(
                          columns=["City","Country","Latitude","Longitude"]
                          )
                          )

In [52]:
population_city_df = population_city_df[[population_city_df.columns[-1]] + list(population_city_df.columns[:-1])]

In [53]:
population_city_df

,city_id,Population,Timestamp_Population
0,1,3878100,2024-12-05
1,2,1964021,2024-12-05
2,3,1510378,2024-12-05


In [54]:
population_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   city_id               3 non-null      int64         
 1   Population            3 non-null      int64         
 2   Timestamp_Population  3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 204.0 bytes


#### 3.2.2.&nbsp; Create "cities_population" table in the database

```sql
CREATE TABLE cities_population (
	city_id INT, -- ID of the cities 
    population_id INT AUTO_INCREMENT, -- Automatically generated ID for each row
    Population INT NOT NULL, -- Population of the city
    Timestamp_Population DATE, -- Time the population is stored
    PRIMARY KEY (population_id),
    FOREIGN KEY (city_id) REFERENCES cities_info(city_id) -- Foreign key to connect each city to its population
);
```

#### 3.2.3.&nbsp; Push the "population_city_df" to the empty "cities_population" table in the database

In [ ]:
population_city_df.to_sql('cities_population',
                if_exists='append',
                con=connection_string,
                index=False)

#### 3.2.4.&nbsp; Retrieving the information from sql to this notebook

In [ ]:
population_city_from_sql = pd.read_sql("cities_population", con=connection_string)
population_city_from_sql

,city_id,population_id,Population,Timestamp_Population
0,1,1,3878100,2024-12-04
1,2,2,1964021,2024-12-04
2,3,3,1510378,2024-12-04


### 3.3.&nbsp; weather_df

#### 3.3.1.&nbsp; Preparing the population table

In [101]:
weather_city_df = weather_df.merge(city_info_from_sql,
                                   left_on='city_name',
                                   right_on='City',
                                   how="left")

weather_city_df.sample(3)

,city_name,forecast_time,outlook,description,temperature,feels_like,temp_min,temp_max,wind_speed,rain_prob,city_id,City,Country,Latitude,Longitude
30,Berlin,2024-12-08 06:00:00,Clouds,overcast clouds,5.49,1.91,5.49,5.49,5.08,0.00,1,Berlin,Germany,52.5200,13.405
72,Hamburg,2024-12-08 12:00:00,Rain,light rain,5.09,2.52,5.09,5.09,3.13,0.95,2,Hamburg,Germany,53.5500,10.000
101,Munich,2024-12-07 03:00:00,Clouds,overcast clouds,4.61,2.96,4.61,4.61,1.96,0.00,3,Munich,Germany,48.1375,11.575


In [ ]:
weather_city_df = (weather_city_df
                      .drop(
                          columns=["city_name","City","Country","Latitude","Longitude"]
                          )
                          )

In [103]:
# Move the last column to the first position
weather_city_df = weather_city_df[[weather_city_df.columns[-1]] + list(weather_city_df.columns[:-1])]

In [104]:
weather_city_df.sample(3)

,city_id,forecast_time,outlook,description,temperature,feels_like,temp_min,temp_max,wind_speed,rain_prob
56,2,2024-12-06 12:00:00,Rain,moderate rain,7.80,4.44,7.80,7.80,6.02,1.00
81,3,2024-12-04 15:00:00,Snow,light snow,2.89,1.42,2.44,2.89,1.59,0.81
45,2,2024-12-05 03:00:00,Clouds,overcast clouds,1.95,-0.80,1.95,1.95,2.59,0.00


In [105]:
weather_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   city_id        120 non-null    int64         
 1   forecast_time  120 non-null    datetime64[ns]
 2   outlook        120 non-null    object        
 3   description    120 non-null    object        
 4   temperature    120 non-null    float64       
 5   feels_like     120 non-null    float64       
 6   temp_min       120 non-null    float64       
 7   temp_max       120 non-null    float64       
 8   wind_speed     120 non-null    float64       
 9   rain_prob      120 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 9.5+ KB


#### 3.3.2.&nbsp; Create "cities_weather" table in the database

```sql
CREATE TABLE cities_weather (
	weather_id INT AUTO_INCREMENT, -- Automatically generated ID for each row
    city_id INT NOT NULL, -- Adding the ID for each city
    forecast_time datetime, 
    outlook VARCHAR(255),
    descriptions VARCHAR(255),
    temperature DECIMAL(4,2),
    feels_like DECIMAL(4,2), 
    temp_min DECIMAL(4,2),
    temp_max DECIMAL(4,2),
    wind_speed DECIMAL(4,2),
    rain_prob DECIMAL(4,2),
    PRIMARY KEY (weather_id),
    FOREIGN KEY (city_id) REFERENCES cities_info(city_id) -- Foreign key to connect each city to its weather
);
```

#### 3.3.3.&nbsp; Push the "weather_city_df" to the empty "cities_weather" table in the database

In [107]:
weather_city_df.to_sql('cities_weather',
                if_exists='append',
                con=connection_string,
                index=False)

120

#### 3.3.4.&nbsp; Retrieving the information from sql to this notebook

In [33]:
weather_city_from_sql = pd.read_sql("cities_weather", con=connection_string)
weather_city_from_sql.sample(3)

,weather_id,city_id,forecast_time,outlook,description,temperature,feels_like,temp_min,temp_max,wind_speed,rain_prob
112,113,3,2024-12-08 12:00:00,Snow,light snow,2.03,-3.55,2.03,2.03,7.62,0.95
63,64,2,2024-12-07 09:00:00,Rain,light rain,6.91,3.12,6.91,6.91,6.54,1.00
15,16,1,2024-12-06 09:00:00,Rain,moderate rain,5.99,2.15,5.99,5.99,6.01,1.00


### 3.4.&nbsp; airports

#### 3.4.1.&nbsp; Preparing the airports table

In [30]:
cities_airports = airports.merge(city_info_from_sql,
                                   left_on='municipalityName',
                                   right_on='City',
                                   how="left")
cities_airports

,airport_icao,airport_name,municipalityName,city_id,City,Country,Latitude,Longitude
0,EDDB,Berlin Brandenburg,Berlin,1,Berlin,Germany,52.5200,13.405
1,EDDH,Hamburg,Hamburg,2,Hamburg,Germany,53.5500,10.000
2,EDDM,Munich,Munich,3,Munich,Germany,48.1375,11.575


In [31]:
cities_airports = (cities_airports
            .drop(
                columns=["municipalityName", "City", "Country", "Latitude", "Longitude"]
                , axis=1)
                )

In [32]:
# Move the last column to the first position
cities_airports = cities_airports[[cities_airports.columns[-1]] + list(cities_airports.columns[:-1])]

In [33]:
cities_airports

,city_id,airport_icao,airport_name
0,1,EDDB,Berlin Brandenburg
1,2,EDDH,Hamburg
2,3,EDDM,Munich


In [34]:
cities_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city_id       3 non-null      int64 
 1   airport_icao  3 non-null      object
 2   airport_name  3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


#### 3.4.2.&nbsp; Create "cities_airport" table in the database

```sql
CREATE TABLE cities_airport (
	airport_id INT AUTO_INCREMENT, -- Automatically generated ID for each row
    city_id INT NOT NULL, -- Adding the ID for each city
    airport_icao VARCHAR(255),
    airport_name VARCHAR(255),
    PRIMARY KEY (airport_id),
    FOREIGN KEY (city_id) REFERENCES cities_info(city_id) -- Foreign key to connect each city to its weather
);
```

#### 3.4.3.&nbsp; Push the "cities_airports" to the empty "cities_airport" table in the database

In [41]:
cities_airports.to_sql('cities_airport',
                if_exists='append',
                con=connection_string,
                index=False)

3

#### 3.4.4.&nbsp; Retrieving the information from sql to this notebook

In [34]:
airports_city_from_sql = pd.read_sql("cities_airport", con=connection_string)
airports_city_from_sql

,airport_id,city_id,airport_icao,airport_name
0,1,1,EDDB,Berlin Brandenburg
1,2,2,EDDH,Hamburg
2,3,3,EDDM,Munich


### 3.5.&nbsp; flight_df

#### 3.5.1.&nbsp; Preparing the airports table

In [123]:
flight_airports = flight_df.merge(cities_airports,
                                   left_on='icao',
                                   right_on='airport_icao',
                                   how="left")
flight_airports.sample(3)

,icao,icao_arrival,arrival_local_time,revised_local_time,flight_number,flight_status,city_id,airport_icao,airport_name
227,EDDH,LEPA,2024-12-04 15:50:00,2024-12-04 15:50:00,EW 7583,Expected,2,EDDH,Hamburg
256,EDDH,OMDB,2024-12-04 19:05:00,2024-12-04 19:05:00,EK 61,Expected,2,EDDH,Hamburg
145,EDDB,EIDW,2024-12-04 18:50:00,2024-12-04 18:50:00,EI 336,Expected,1,EDDB,Berlin Brandenburg


In [125]:
flight_airports = (flight_airports
                      .drop(
                          columns=["airport_icao","airport_name"]
                          )
                          )

In [126]:
flight_airports = flight_airports[[flight_airports.columns[-1]] + list(flight_airports.columns[:-1])]

In [127]:
flight_airports.sample(3)

,city_id,icao,icao_arrival,arrival_local_time,revised_local_time,flight_number,flight_status
145,1,EDDB,EIDW,2024-12-04 18:50:00,2024-12-04 18:50:00,EI 336,Expected
357,3,EDDM,EGLL,2024-12-04 11:35:00,2024-12-04 11:40:00,LH 2471,Expected
385,3,EDDM,EGBB,2024-12-04 13:35:00,2024-12-04 13:35:00,VL 2509,Expected


In [128]:
flight_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   city_id             556 non-null    int64         
 1   icao                556 non-null    object        
 2   icao_arrival        548 non-null    object        
 3   arrival_local_time  556 non-null    datetime64[ns]
 4   revised_local_time  553 non-null    datetime64[ns]
 5   flight_number       556 non-null    object        
 6   flight_status       556 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 30.5+ KB


#### 3.5.2.&nbsp; Create "cities_flight" table in the database

```sql
CREATE TABLE cities_flight (
	flights_id INT AUTO_INCREMENT, -- Automatically generated ID for each row
    city_id INT NOT NULL, -- Adding the ID for each city
    icao VARCHAR(255),
    icao_arrival VARCHAR(255),
    arrival_local_time datetime, 
    revised_local_time datetime, 
    flight_number VARCHAR(255),
    flight_status VARCHAR(255),
    PRIMARY KEY (flights_id),
    FOREIGN KEY (city_id) REFERENCES cities_info(city_id) -- Foreign key to connect each city to its weather
);
```

#### 3.5.3.&nbsp; Push the "flight_airports" to the empty "cities_flight" table in the database

In [129]:
flight_airports.to_sql('cities_flight',
                if_exists='append',
                con=connection_string,
                index=False)

556

#### 3.5.4.&nbsp; Retrieving the information from sql to this notebook

In [35]:
flight_airports_from_sql = pd.read_sql("cities_flight", con=connection_string)
flight_airports_from_sql.sample(3)

,flights_id,city_id,icao,icao_arrival,arrival_local_time,revised_local_time,flight_number,flight_status
202,203,2,EDDH,EDDM,2024-12-04 12:10:00,2024-12-04 12:10:00,VL 2062,Expected
536,537,3,EDDM,ZBAA,2024-12-04 19:15:00,2024-12-04 19:15:00,LH 723,Expected
399,400,3,EDDM,LJLJ,2024-12-04 14:05:00,2024-12-04 14:05:00,LH 1697,Expected


---
## 4.&nbsp; Pushing the static data from Python to the Google Cloud 
This can be done the same way we did with our local instance. However, don't forget to change your **connection string**. You will need to change the **schema name**, **table name**, potentially **your password**, but definitely **the variable** and **the host**.

First: import from local

In [39]:
# Save your password in a .py file called safe.
from safe import sql_password

In [36]:
schema = "CityDatabase"
host = "127.0.0.1"
user = "root"
password = sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [37]:
city_info_GC = pd.read_sql("cities_info", con=connection_string)
city_info_GC

,city_id,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


In [56]:
city_info_GC = (city_info_GC
                      .drop(
                          columns=["city_id"]
                          )
                          )

In [57]:
city_info_GC

,City,Country,Latitude,Longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


In [61]:
population_city_GC = pd.read_sql("cities_population", con=connection_string)
population_city_GC

,city_id,population_id,Population,Timestamp_Population
0,1,1,3878100,2024-12-04
1,2,2,1964021,2024-12-04
2,3,3,1510378,2024-12-04


In [65]:
airports_city_GC = pd.read_sql("cities_airport", con=connection_string)
airports_city_GC

,airport_id,city_id,airport_icao,airport_name
0,1,1,EDDB,Berlin Brandenburg
1,2,2,EDDH,Hamburg
2,3,3,EDDM,Munich


Second: export to the google cloud

In [44]:
# Save your password in a .py file called safe.
from safe import sql_password

In [40]:
schema = "gans_schema"
host = "34.78.116.238"
user = "root"
password = sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [59]:
city_info_GC.to_sql('cities_info',
                 if_exists='append',
                  con=connection_string,
                  index=False)

3

In [63]:
population_city_GC.to_sql('cities_population',
                if_exists='append',
                con=connection_string,
                index=False)

3

In [67]:
airports_city_GC.to_sql('cities_airport',
                if_exists='append',
                con=connection_string,
                index=False)

3